# TASK1

In [39]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report


In [3]:
from google.colab import files
 
 
uploaded = files.upload()

Saving DS3_C3_S2_TweetTest_Data_Challenge.csv to DS3_C3_S2_TweetTest_Data_Challenge (1).csv
Saving DS3_C3_S2_TweetTrain_Data_Challenge.csv to DS3_C3_S2_TweetTrain_Data_Challenge (1).csv


In [4]:
Train = pd.read_csv('DS3_C3_S2_TweetTrain_Data_Challenge.csv')

In [5]:
Test = pd.read_csv('DS3_C3_S2_TweetTest_Data_Challenge.csv')

In [6]:
test = Test.loc[:,['text','sentiment']]

In [7]:
test.head()

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive


In [8]:
Train.dropna(inplace=True)

In [9]:
Train.isna().sum()

textID              0
text                0
selected_text       0
sentiment           0
Time of Tweet       0
Age of User         0
Country             0
Population -2020    0
Land Area (Km�)     0
Density (P/Km�)     0
dtype: int64

In [10]:
train= Train.loc[:,['text','sentiment']]
train.head()

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


# TASK2 : Preprocessing

In [11]:
# Downloads the data.
import nltk
nltk.download('stopwords')


# Using the stopwords.
from nltk.corpus import stopwords

# Initialize the stopwords
Stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
train['text'] = train['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
test['text'] = test['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


In [13]:
train['text'] = train['text'].apply(lambda x: ' '.join(x for x in x.split() if x.lower() not in  Stop_words))
test['text'] = test['text'].apply(lambda x: ' '.join(x for x in x.split() if x.lower() not in  Stop_words))


In [14]:
train.head()

,text,sentiment
0,Id responded going,neutral
1,Sooo SAD miss San Diego,negative
2,boss bullying,negative
3,interview leave alone,negative
4,Sons couldnt put releases already bought,negative


In [15]:
test.head()

,text,sentiment
0,Last session day httptwitpiccom67ezh,neutral
1,Shanghai also really exciting precisely skyscr...,positive
2,Recession hit Veronique Branquinho quit compan...,negative
3,happy bday,positive
4,httptwitpiccom4w75p like,positive


# TASK3 : Word2Vec

In [16]:
vec =TfidfVectorizer()
X = vec.fit_transform(train.text).toarray()

In [17]:
X.shape

(27480, 29410)

In [18]:
Y = pd.get_dummies(train.sentiment)

In [19]:
Y.shape

(27480, 3)

In [20]:
model = LogisticRegression(max_iter=200)

In [21]:
ovr = OneVsRestClassifier(model)

In [22]:
ovr.fit(X,Y)

OneVsRestClassifier(estimator=LogisticRegression(max_iter=200))

## TASK 4 : Prediction

In [24]:
X_test = vec.transform(test.text).toarray()

In [25]:
X_test.shape

(3534, 29410)

In [27]:
Y_test = pd.get_dummies(test.sentiment)
Y_test.shape

(3534, 3)

In [28]:
y_pred = ovr.predict(X_test)
y_pred

array([[0, 1, 0],
       [0, 0, 1],
       [1, 1, 0],
       ...,
       [0, 0, 0],
       [0, 0, 1],
       [0, 0, 1]])

In [36]:
y_pred = pd.DataFrame(y_pred, columns = Y_test.columns)

In [35]:
pd.concat([y_pred.head(30).reset_index().add_prefix('Predict_'), Y_test.head(30).reset_index().add_prefix('Actual_')], axis=1).drop(['Predict_index','Actual_index'],axis=1)

,Predict_negative,Predict_neutral,Predict_positive,Actual_negative,Actual_neutral,Actual_positive
0,0,1,0,0,1,0
1,0,0,1,0,0,1
2,1,1,0,1,0,0
3,0,0,1,0,0,1
4,0,0,0,0,0,1
5,0,0,1,0,0,1
6,1,0,0,1,0,0
7,0,0,0,1,0,0
8,0,1,0,0,1,0
9,0,1,0,0,1,0


In [38]:
print(classification_report(Y_test,y_pred ))

              precision    recall  f1-score   support

           0       0.80      0.52      0.63      1001
           1       0.68      0.59      0.63      1430
           2       0.86      0.63      0.72      1103

   micro avg       0.76      0.58      0.66      3534
   macro avg       0.78      0.58      0.66      3534
weighted avg       0.77      0.58      0.66      3534
 samples avg       0.58      0.58      0.58      3534



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
